# Team Operations: Reset, Stop, Resume and Abort

In [16]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENROUTER_API_KEY")

model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model = "meta-llama/llama-3.3-70b-instruct:free",
    api_key=api_key,
    model_info={
        "family":"meta-llama",
        "vision" : True,
        "function_calling":True,
        "json_output":False,
    }
)

from autogen_ext.models.ollama import OllamaChatCompletionClient

client = OllamaChatCompletionClient(model="gemma2:latest")

In [18]:
from autogen_agentchat.agents import AssistantAgent

add_1_agent_first = AssistantAgent(
    name = "add_1_agent_first",
    model_client=client,
    system_message="Add 1 to the number and just give out resultant number. Start with 0 if nothing is provided."
)

add_1_agent_second = AssistantAgent(
    name = "add_1_agent_second",
    model_client=client,
    system_message="Add 1 to the number and just give out resultant number."
)

add_1_agent_third = AssistantAgent(
    name = "add_1_agent_third",
    model_client=client,
    system_message="Add 1 to the number and just give out resultant number."
)

In [19]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third],
    max_turns= 3,
)

In [20]:
from autogen_agentchat.ui import Console

await Console(team.run_stream(task ="First number 0"))


---------- TextMessage (user) ----------
First number 0


---------- TextMessage (add_1_agent_first) ----------
1 

---------- TextMessage (add_1_agent_second) ----------
1 

---------- TextMessage (add_1_agent_third) ----------
2 



TaskResult(messages=[TextMessage(id='bb5e7f56-a77f-4af9-889a-cfa16c0e10cc', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 7, 8, 17, 51, 714776, tzinfo=datetime.timezone.utc), content='First number 0', type='TextMessage'), TextMessage(id='8be6837d-acee-46ec-bf17-bd1a06f187eb', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=35, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 10, 7, 8, 18, 10, 846090, tzinfo=datetime.timezone.utc), content='1 \n', type='TextMessage'), TextMessage(id='069476ed-cd5e-4751-9521-190f7672ee76', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=30, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 10, 7, 8, 18, 13, 214682, tzinfo=datetime.timezone.utc), content='1 \n', type='TextMessage'), TextMessage(id='608e2c0a-e0a0-455c-b25e-8f96e6e172ec', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=33, completion_tokens=4), metada

# Resuming a Team

teams are stateful and maintains the conversation history and context after each run, unless you reset the team.

We can resume a team to continue from where it left off by calling the run() or run_stream() method without a raw task.


In [21]:
# ye previous number ke aage se continue krega add krna
await Console(team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
3  

---------- TextMessage (add_1_agent_second) ----------
4 

---------- TextMessage (add_1_agent_third) ----------
5 



TaskResult(messages=[TextMessage(id='386d6516-8cd1-42cf-8d2e-b65d52a9d29c', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=54, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 10, 7, 8, 18, 23, 172406, tzinfo=datetime.timezone.utc), content='3  \n', type='TextMessage'), TextMessage(id='47d1017d-58ae-4862-921a-10deb8e9cd54', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=49, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 10, 7, 8, 18, 27, 838172, tzinfo=datetime.timezone.utc), content='4 \n', type='TextMessage'), TextMessage(id='2ca8e16f-96bd-49bf-8568-29bd9af9ae31', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=52, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 10, 7, 8, 18, 31, 939521, tzinfo=datetime.timezone.utc), content='5 \n', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

In [22]:
await Console(team.run_stream(task="What was the larget number you got in result"))

---------- TextMessage (user) ----------
What was the larget number you got in result
---------- TextMessage (add_1_agent_first) ----------
The largest result I got was 5.  

Let me know if you'd like to continue playing! 😊 

---------- TextMessage (add_1_agent_second) ----------
Okay, let's keep going! 😄

Give me the next number.   

---------- TextMessage (add_1_agent_third) ----------
You're right! My apologies, I seem to have gotten confused. 

I'll keep track better this time.  Please give me the next number. 👍 



TaskResult(messages=[TextMessage(id='94673bf0-0605-4f31-b097-67ed856094fe', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 7, 8, 18, 31, 957893, tzinfo=datetime.timezone.utc), content='What was the larget number you got in result', type='TextMessage'), TextMessage(id='e4bdd492-04c2-4a80-9e76-0d36922b91e5', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=83, completion_tokens=27), metadata={}, created_at=datetime.datetime(2025, 10, 7, 8, 18, 43, 910823, tzinfo=datetime.timezone.utc), content="The largest result I got was 5.  \n\nLet me know if you'd like to continue playing! 😊 \n", type='TextMessage'), TextMessage(id='9e0f612b-fc4a-4da7-8665-5b7ddca1bad6', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=102, completion_tokens=19), metadata={}, created_at=datetime.datetime(2025, 10, 7, 8, 18, 56, 421028, tzinfo=datetime.timezone.utc), content="Okay, let's keep going! 😄\n\nGive me the next number.   \n", type=

## Reset our team


In [23]:
await team.reset()    ## intrnally call on_reset() to reset all agents

In [24]:
await Console(team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
0 

---------- TextMessage (add_1_agent_second) ----------
1 

---------- TextMessage (add_1_agent_third) ----------
2  



TaskResult(messages=[TextMessage(id='c97e2e42-ed7a-4bb2-9530-4e815cc9aa05', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=32, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 10, 7, 8, 20, 46, 863818, tzinfo=datetime.timezone.utc), content='0 \n', type='TextMessage'), TextMessage(id='c40a0f3d-7eb5-442f-b16d-c9923202595a', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=26, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 10, 7, 8, 20, 48, 891182, tzinfo=datetime.timezone.utc), content='1 \n', type='TextMessage'), TextMessage(id='e86516fa-a977-4234-95fa-08beec6e2259', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 10, 7, 8, 20, 50, 985870, tzinfo=datetime.timezone.utc), content='2  \n', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')